TiDE (time series dense encoder implementation)

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path('../../.').resolve()))

In [10]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
import json


from src.models.tide.TiDE import TiDEModel
from src.data.diabetes_datasets.data_loader import get_loader
from src.data.models import ColumnNames

In [3]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cpu


In [ ]:
# Load data using the data loader
train_loader = get_loader(
    data_source_name="kaggle_brisT1D", dataset_type="train", use_cached=False
)
test_loader = get_loader(
    data_source_name="kaggle_brisT1D", dataset_type="test", use_cached=False
)

train_data = train_loader.train_data
test_data = test_loader.test_data



2026-01-13T18:44:35 - ============================================================
2026-01-13T18:44:35 - Beginning data loading process with the following parmeters:
2026-01-13T18:44:35 - 	Dataset: kaggle_brisT1D - train
2026-01-13T18:44:35 - 	Columns: None
2026-01-13T18:44:35 - 	Generic patient start date: 2024-01-01 00:00:00
2026-01-13T18:44:35 - 	Number of validation days: 20
2026-01-13T18:44:35 - 	In parallel with up to 3 workers.

2026-01-13T18:44:35 - Processed cache not found or not used, processing raw data and saving to cache...
2026-01-13T18:44:35 - Raw data for kaggle_brisT1D already exists in cache
2026-01-13T18:44:41 - _process_raw_train_data: Processing train data. This may take a while...
2026-01-13T18:44:41 - Processing 9 patients:
2026-01-13T18:44:52 - processed_results: Successfully processed patient p01
2026-01-13T18:45:11 - processed_results: Successfully processed patient p03
2026-01-13T18:45:11 - processed_results: Successfully processed patient p02
2026-01-13T18:

In [114]:
from torch.utils.data import Dataset

class TiDEWindowedDataset(Dataset):
    """Dataset for TiDE model with proper windowing for time series prediction"""
    
    def __init__(self, data_loader_instance, lookback_steps=480, horizon_steps=120, 
                 dataset_type='train', stride=60):
        """
        Args:
            data_loader_instance: Instance of BrisT1DDataLoader
            lookback_steps: Number of timesteps for lookback (480 = 8 hours if 1 min resolution)
            horizon_steps: Number of timesteps for prediction (120 = 2 hours)
            dataset_type: 'train', 'validation', or 'test'
            stride: Step size for sliding window (60 = 1 hour)
        """
        self.lookback = lookback_steps
        self.horizon = horizon_steps
        self.stride = stride
        
        # Get the appropriate data
        if dataset_type == 'train' and data_loader_instance.train_data:
            self.data_dict = data_loader_instance.train_data
        elif dataset_type == 'validation' and data_loader_instance.validation_data:
            self.data_dict = data_loader_instance.validation_data
        else:
            self.data_dict = data_loader_instance.processed_data
        
        # Priority features for glucose prediction
        self.feature_cols = ['bg_mM', 'food_g', 'dose_units', 'iob', 'cob']
        self.target_col = 'bg_mM'  # What we're predicting
        
        # Validate columns exist
        first_value = next(iter(self.data_dict.values()))
        
        # Handle nested dict structure (get first DataFrame from nested dict)
        if isinstance(first_value, dict):
            first_df = next(iter(first_value.values()))
        else:
            first_df = first_value
        
        self.feature_cols = [col for col in self.feature_cols if col in first_df.columns]
        
        # Prepare valid windows
        self.windows = []
        self.prepare_windows()
        
        print(f"Created TiDE dataset:")
        print(f"  - Lookback: {lookback_steps} steps")
        print(f"  - Horizon: {horizon_steps} steps")
        print(f"  - Features: {self.feature_cols}")
        print(f"  - Total windows: {len(self.windows)}")
    
    def prepare_windows(self):
        """Prepare all valid windows from all patients, handling nested dict structures"""
        for patient_id, patient_data in self.data_dict.items():
            # Handle nested dictionary structure (e.g., test data: {patient_id: {session_id: df}})
            if isinstance(patient_data, dict):
                for session_id, patient_df in patient_data.items():
                    self._process_patient_df(patient_df, f"{patient_id}_{session_id}")
            # Handle flat dictionary structure (e.g., train data: {patient_id: df})
            elif isinstance(patient_data, pd.DataFrame):
                self._process_patient_df(patient_data, patient_id)

    def _process_patient_df(self, patient_df, patient_key):
        """Process a single patient dataframe and create windows"""
        n_samples = len(patient_df)
        min_length = self.lookback + self.horizon
        if n_samples >= min_length:
            windows_count = 0
            for i in range(self.lookback, n_samples - self.horizon + 1, self.stride):
                self.windows.append((patient_key, i))
                windows_count += 1
            if windows_count > 0:
                print(f"  Patient {patient_key}: {len(patient_df)} samples -> {windows_count} windows")
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        patient_id, end_lookback_idx = self.windows[idx]
        patient_df = self.data_dict[patient_id]
        
        # Get lookback window (input sequence)
        lookback_start = end_lookback_idx - self.lookback
        lookback_data = patient_df.iloc[lookback_start:end_lookback_idx]
        
        # Get prediction horizon (target sequence)
        horizon_data = patient_df.iloc[end_lookback_idx:end_lookback_idx + self.horizon]
        
        # Extract features
        seq_x = lookback_data[self.feature_cols].values.astype(np.float32)
        
        # For target, typically just predict glucose (bg_mM)
        # You can modify this based on what TiDE expects
        seq_y = horizon_data[[self.target_col]].values.astype(np.float32)
        
        # Handle NaN values
        seq_x = np.nan_to_num(seq_x, nan=0.0)
        seq_y = np.nan_to_num(seq_y, nan=0.0)
        
        # Convert to tensors
        seq_x = torch.tensor(seq_x, dtype=torch.float32)
        seq_y = torch.tensor(seq_y, dtype=torch.float32)
        
        # Time features (can be enhanced with hour of day, day of week, etc.)
        seq_x_mark = torch.arange(self.lookback, dtype=torch.float32).reshape(-1, 1)
        seq_y_mark = torch.arange(self.horizon, dtype=torch.float32).reshape(-1, 1)
        
        return seq_x, seq_y, seq_x_mark, seq_y_mark

In [173]:
train_data = TiDEWindowedDataset(train_loader, dataset_type='train')
val_data = TiDEWindowedDataset(train_loader, dataset_type='validation')

  Patient p01: 8711 samples -> 136 windows
  Patient p03: 26423 samples -> 431 windows
  Patient p02: 26423 samples -> 431 windows
  Patient p04: 24983 samples -> 407 windows
  Patient p05: 8808 samples -> 137 windows
  Patient p06: 8791 samples -> 137 windows
  Patient p12: 26371 samples -> 430 windows
  Patient p11: 25559 samples -> 416 windows
  Patient p10: 25803 samples -> 421 windows
Created TiDE dataset:
  - Lookback: 480 steps
  - Horizon: 120 steps
  - Features: ['bg_mM', 'food_g', 'dose_units', 'iob', 'cob']
  - Total windows: 2946
  Patient p01: 8711 samples -> 136 windows
  Patient p03: 26423 samples -> 431 windows
  Patient p02: 26423 samples -> 431 windows
  Patient p04: 24983 samples -> 407 windows
  Patient p05: 8808 samples -> 137 windows
  Patient p06: 8791 samples -> 137 windows
  Patient p12: 26371 samples -> 430 windows
  Patient p11: 25559 samples -> 416 windows
  Patient p10: 25803 samples -> 421 windows
Created TiDE dataset:
  - Lookback: 480 steps
  - Horizon: 

In [178]:
def compute_metrics(predictions, targets):
    """Compute RMSE, MAE, MAPE for forecast evaluation."""
    y_pred = np.asarray(predictions).flatten()
    y_true = np.asarray(targets).flatten()
    
    # Filter out near-zero targets to avoid MAPE explosion
    mask = np.abs(y_true) > 0.1  # Only use targets > 0.1 mmol/L
    y_pred_filtered = y_pred[mask]
    y_true_filtered = y_true[mask]
    
    if len(y_true_filtered) == 0:
        mape = np.nan
    else:
        mape = float(np.mean(np.abs((y_pred_filtered - y_true_filtered) / y_true_filtered)) * 100)
    
    return {
        "rmse": float(root_mean_squared_error(y_true, y_pred)),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "mape": mape,
        "samples_used": len(y_true_filtered),
        "samples_filtered": len(y_true) - len(y_true_filtered)
    }

In [118]:
from src.models.tide.parse_args import *
from torch.utils.data import DataLoader

In [130]:
train_dataloader = DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=0
)

val_dataloader = DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=0
)

test_dataloader = DataLoader(
    test_data,
    batch_size=32,
    shuffle=False,
    num_workers=0
)
    
for seq_x, seq_y, seq_x_mark, seq_y_mark in train_dataloader:
    print("Sample batch shapes:")
    print(f"  seq_x (features):     {seq_x.shape}")
    print(f"  seq_y (target):       {seq_y.shape}")
    print(f"  seq_x_mark (time):    {seq_x_mark.shape}")
    print(f"  seq_y_mark (time):    {seq_y_mark.shape}")

    sizes = {
    'lookback': (seq_y.shape[1], seq_y.shape[2]),
    'attr': (seq_x.shape[1], seq_x.shape[2]),
    'dynCov': (seq_y_mark.shape[1], seq_y_mark.shape[2])
}
    print(f"  lookback: {sizes['lookback']}")
    print(f"  attr: {sizes['attr']}")
    print(f"  dynCov: {sizes['dynCov']}")
    break

Sample batch shapes:
  seq_x (features):     torch.Size([32, 480, 5])
  seq_y (target):       torch.Size([32, 120, 1])
  seq_x_mark (time):    torch.Size([32, 480, 1])
  seq_y_mark (time):    torch.Size([32, 120, 1])
  lookback: (120, 1)
  attr: (480, 5)
  dynCov: (120, 1)


In [ ]:
TARGET_COL = 'bg_mM'

In [122]:
class Config:
    """Configuration class for TiDE model and training."""

    # Data parameters
    target: str = TARGET_COL  # Blood glucose in mmol/L

    # Sequence parameters (in 5-minute intervals)
    # Lookback: 6 hours = 72 intervals of 5 minutes
    # Prediction: 1 hour = 12 intervals of 5 minutes
    seq_len: int = 72        # Input sequence length (6 hours)
    lookback_len: int = 72   # Lookback length for decoder
    pred_len: int = 12       # Prediction horizon (1 hour)

    # Model parameters
    feat_size: int = 4           # Feature projection size
    hidden_size: int = 128       # Hidden layer size
    num_encoder_layers: int = 2  # Number of encoder layers
    num_decoder_layers: int = 2  # Number of decoder layers
    decoder_output_dim: int = 8  # Decoder output dimension per timestep
    temporal_decoder_hidden: int = 64  # Temporal decoder hidden size
    drop_prob: float = 0.3       # Dropout probability

    # Training parameters
    batch_size: int = 32
    epochs: int = 50
    lr: float = 1e-4
    patience: int = 10  # Early stopping patience

    # Data split
    train_ratio: float = 0.7
    val_ratio: float = 0.15
    test_ratio: float = 0.15

    # Device
    use_cuda: bool = True

    def __repr__(self):
        return json.dumps({k: v for k, v in self.__class__.__dict__.items()
                          if not k.startswith('_')}, indent=2, default=str)

config = Config()
print("Configuration:")
print(config)

Configuration:
{
  "target": "bg_mM",
  "seq_len": 72,
  "lookback_len": 72,
  "pred_len": 12,
  "feat_size": 4,
  "hidden_size": 128,
  "num_encoder_layers": 2,
  "num_decoder_layers": 2,
  "decoder_output_dim": 8,
  "temporal_decoder_hidden": 64,
  "drop_prob": 0.3,
  "batch_size": 32,
  "epochs": 50,
  "lr": 0.0001,
  "patience": 10,
  "train_ratio": 0.7,
  "val_ratio": 0.15,
  "test_ratio": 0.15,
  "use_cuda": true
}


In [131]:
model = TiDEModel(sizes, config)
model = model.to(device)
print(model)

TiDEModel(
  (featproj): ResidualBlock(
    (fc1): Linear(in_features=1, out_features=128, bias=True)
    (relu1): ReLU(inplace=True)
    (fc2): Linear(in_features=128, out_features=4, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
    (shortcut): Linear(in_features=1, out_features=4, bias=True)
  )
  (encoder): TiDEEncoder(
    (blocks): ModuleList(
      (0): ResidualBlock(
        (fc1): Linear(in_features=2952, out_features=128, bias=True)
        (relu1): ReLU(inplace=True)
        (fc2): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (shortcut): Linear(in_features=2952, out_features=128, bias=True)
      )
      (1): ResidualBlock(
        (fc1): Linear(in_features=128, out_features=128, bias=True)
        (relu1): ReLU(inplace=True)
        (fc2): Linear(in_features=128, out_f

In [158]:
from tqdm import tqdm
import time

In [165]:
def train_epoch(model, train_loader, optimizer, criterion, device, epoch):
    """Train for one epoch."""
    model.train()
    train_mse_loss = 0.
    train_mae_loss = 0.

    pbar = tqdm(train_loader, desc=f'Epoch {epoch}')

    for seq_x, seq_y, seq_x_mark, seq_y_mark in train_loader:
        seq_x = seq_x.float().to(device)
        seq_y = seq_y.float().to(device)
        seq_x_mark = seq_x_mark.float().to(device)
        seq_y_mark = seq_y_mark.float().to(device)

        # Keep seq_y as 3D: [batch, seq_len, features]
        # Do NOT squeeze the last dimension
        
        # Update model batch size for variable batch sizes
        current_batch_size = seq_x.shape[0]
        model.batch_size = current_batch_size
       
        optimizer.zero_grad()

        pred, ans = model(seq_x, seq_y, seq_x_mark, seq_y_mark)

        loss = criterion(pred, ans)
        mae_loss = torch.mean(torch.abs(pred - ans))

        loss.backward()
        optimizer.step()

        train_mse_loss += loss.item()
        train_mae_loss += mae_loss.item()

        pbar.set_postfix({
            'MSE': f'{loss.item():.4f}',
            'MAE': f'{mae_loss.item():.4f}'
        })

    avg_mse = train_mse_loss / len(train_loader)
    avg_mae = train_mae_loss / len(train_loader)

    return avg_mse, avg_mae

In [166]:
def validate(model, val_loader, criterion, device):
    """Validate the model on validation set."""
    model.eval()
    val_mse_loss = 0.
    val_mae_loss = 0.

    with torch.no_grad():
        for seq_x, seq_y, seq_x_mark, seq_y_mark in val_loader:
            seq_x = seq_x.float().to(device)
            seq_y = seq_y.float().to(device)
            seq_x_mark = seq_x_mark.float().to(device)
            seq_y_mark = seq_y_mark.float().to(device)

            # Keep seq_y as 3D: [batch, seq_len, features]
            
            # Update model batch size for variable batch sizes
            current_batch_size = seq_x.shape[0]
            model.batch_size = current_batch_size

            pred, ans = model(seq_x, seq_y, seq_x_mark, seq_y_mark)

            loss = criterion(pred, ans)
            mae_loss = torch.mean(torch.abs(pred - ans))

            val_mse_loss += loss.item()
            val_mae_loss += mae_loss.item()

    val_mse_loss /= len(val_loader)
    val_mae_loss /= len(val_loader)

    model.train()
    return val_mse_loss, val_mae_loss

In [182]:
def train_model(model, train_loader, val_loader, config, device):
    """Train the TiDE model."""
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    criterion = torch.nn.MSELoss(reduction='mean')

    history = {
        'epoch': [],
        'train_mse': [],
        'train_mae': [],
        'val_mse': [],
        'val_mae': []
    }

    print(f"\nStarting training for {config.epochs} epochs...")
    print(f"Training samples: {len(train_loader.dataset)}")
    print(f"Validation samples: {len(val_loader.dataset)}")
    print(f"Batch size: {config.batch_size}")
    print(f"Device: {device}\n")

    start_time = time.time()

    for epoch in range(1, config.epochs + 1):
        # Train
        train_mse, train_mae = train_epoch(
            model, train_loader, optimizer, criterion, device, epoch
        )

        # Validate
        ## val_mse, val_mae = validate(model, val_loader, criterion, device)

        # Log
        history['epoch'].append(epoch)
        history['train_mse'].append(train_mse)
        history['train_mae'].append(train_mae)
        # history['val_mse'].append(val_mse)
        # history['val_mae'].append(val_mae)

        print(f"\nEpoch {epoch}/{config.epochs}:")
        print(f"  Train - MSE: {train_mse:.4f}, MAE: {train_mae:.4f}")
        ## print(f"  Val   - MSE: {val_mse:.4f}, MAE: {val_mae:.4f}")


    total_time = time.time() - start_time
    print(f"\nTraining completed in {total_time:.2f} seconds")
    history['training_time'] = [total_time]

    return history

print("Training functions defined!")

Training functions defined!


In [168]:
trained_model = train_model(model, train_dataloader, val_dataloader, config, device)


Starting training for 50 epochs...
Training samples: 2946
Validation samples: 2946
Batch size: 32
Device: cpu

























































































































































































































































































































































































































































































Epoch 1:   0%|          | 0/93 [00:05<?, ?it/s, MSE=8.3819, MAE=2.2106]



Epoch 1/50:
  Train - MSE: 14.8361, MAE: 2.8379
  Val   - MSE: 14.1437, MAE: 2.7527


















































































Epoch 1:   0%|          | 0/93 [01:28<?, ?it/s]






































































































































































































































































































































































































Epoch 2:   0%|          | 0/93 [00:05<?, ?it/s, MSE=0.3229, MAE=0.4420]



Epoch 2/50:
  Train - MSE: 14.0459, MAE: 2.7399
  Val   - MSE: 13.9285, MAE: 2.7335


...


Epoch 48/50:
  Train - MSE: 12.4484, MAE: 2.5829
  Val   - MSE: 12.3781, MAE: 2.5707


Epoch 49:   0%|          | 0/93 [00:07<?, ?it/s, MSE=0.8932, MAE=0.8153] 



Epoch 49/50:
  Train - MSE: 12.3926, MAE: 2.5735
  Val   - MSE: 12.3738, MAE: 2.5663


Epoch 50:   0%|          | 0/93 [00:07<?, ?it/s, MSE=9.1597, MAE=2.0403] 


Epoch 50/50:
  Train - MSE: 12.4590, MAE: 2.5820
  Val   - MSE: 12.3570, MAE: 2.5674

Training completed in 344.61 seconds


In [175]:
def evaluate_model(model, test_loader, device, scaler=None):
    """Evaluate the model on test set."""
    model.eval()

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for seq_x, seq_y, seq_x_mark, seq_y_mark in tqdm(test_loader, desc='Evaluating'):
            seq_x = seq_x.float().to(device)
            seq_y = seq_y.float().to(device)
            seq_x_mark = seq_x_mark.float().to(device)
            seq_y_mark = seq_y_mark.float().to(device)

            # Update model batch size for variable batch sizes
            current_batch_size = seq_x.shape[0]
            model.batch_size = current_batch_size

            pred, ans = model(seq_x, seq_y, seq_x_mark, seq_y_mark)

            all_preds.append(pred.cpu().numpy())
            all_targets.append(ans.cpu().numpy())

    # Handle empty predictions
    if len(all_preds) == 0:
        print("No predictions made - test dataset may be empty")
        return None

    # Concatenate all batches
    preds = np.concatenate(all_preds, axis=0)
    targets = np.concatenate(all_targets, axis=0)

    # Inverse transform if scaler provided
    if scaler is not None:
        preds_flat = preds.reshape(-1, 1)
        targets_flat = targets.reshape(-1, 1)

        preds_flat = scaler.inverse_transform(preds_flat)
        targets_flat = scaler.inverse_transform(targets_flat)

        preds = preds_flat.reshape(preds.shape)
        targets = targets_flat.reshape(targets.shape)

    # Compute metrics
    metrics = compute_metrics(preds.flatten(), targets.flatten())

    print("\n" + "="*50)
    print("Test Set Evaluation Results")
    print("="*50)
    for metric_name, value in metrics.items():
        if metric_name == 'MAPE':
            print(f"{metric_name}: {value:.2f}%")
        else:
            print(f"{metric_name}: {value:.4f}")

    return {
        'predictions': preds,
        'targets': targets,
        'metrics': metrics
    }

In [179]:
results = evaluate_model(
    model,
    val_dataloader,
    device
)

Evaluating: 100%|██████████| 93/93 [00:02<00:00, 42.80it/s]


Test Set Evaluation Results
rmse: 3.5316
mae: 2.5846
mape: 29.9998
samples_used: 33967.0000
samples_filtered: 1385.0000


In [1]:
def plot_predictions(results, num_samples=5, pred_horizon=12):
    """Plot sample predictions vs ground truth."""
    preds = results['predictions']
    targets = results['targets']

    # Select random samples
    indices = np.random.choice(len(preds), min(num_samples, len(preds)), replace=False)

    fig, axes = plt.subplots(num_samples, 1, figsize=(12, 3*num_samples))
    if num_samples == 1:
        axes = [axes]

    time_steps = np.arange(pred_horizon) * 5  # Convert to minutes

    for i, idx in enumerate(indices):
        pred = preds[idx].flatten()
        target = targets[idx].flatten()

        axes[i].plot(time_steps, target, 'b-o', label='Actual BG', markersize=4)
        axes[i].plot(time_steps, pred, 'r--s', label='Predicted BG', markersize=4)
        axes[i].set_xlabel('Minutes ahead')
        axes[i].set_ylabel('Blood Glucose (mmol/L)')
        axes[i].set_title(f'Sample {idx}: Blood Glucose Prediction')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

        # Add reference lines for hypo/hyper glycemia
        axes[i].axhline(y=3.9, color='orange', linestyle=':', alpha=0.7)
        axes[i].axhline(y=10.0, color='red', linestyle=':', alpha=0.7)

    plt.tight_layout()
    plt.show()

plot_predictions(results, num_samples=5, pred_horizon=config.pred_len)

NameError: name 'results' is not defined